# Week 8 - Semantic Networks - Network Filter

In [44]:
#All these packages need to be installed from pip

import nltk #For POS tagging
import sklearn #For generating some matrices
import pandas as pd#For DataFrames
import numpy as np #For arrays
import matplotlib.pyplot as plt #For plotting
import seaborn #MAkes the plots look nice
import IPython.display #For displaying images
import networkx
%matplotlib inline

import sys
#import subprocess
import re

import pickle #if you want to save layouts
import os
import lucem_illud #pip install -U git+git://github.com/Computational-Content-Analysis-2018/lucem_illud.git

In [11]:
def getmatrix(stacked,A):
    influence_matrix = [[0 for i in range(A)] for j in range(A)]
    for row in stacked.iteritems():
        from_ = int(row[0].split('.')[1])-1
        to_ = int(row[0].split('.')[2])-1
        value = float(row[1])
        influence_matrix[from_][to_]=value
    df_ = pd.DataFrame(influence_matrix) 
    
    df_ =df_.rename(index = id_person)
    df_ =df_.rename(columns = id_person)
    return df_

## First Load the example data and tokenzie sentences

In [121]:
import itertools as it
import networkx as nx
import matplotlib

# Load data

In [70]:
redditDF = pd.read_csv('./data/reddit.csv', index_col = 0)
redditDF = redditDF.sort_values('score')

# Get POS tagged_sentences 

In [79]:
redditTopScores = redditDF.sort_values('score')[-100:]
redditTopScores['sentences'] = redditTopScores['text'].apply(lambda x: [nltk.word_tokenize(s) for s in nltk.sent_tokenize(x)])
redditTopScores.index = range(len(redditTopScores) - 1, -1,-1) #Reindex to make things nice in the future
redditTopScores['normalized_sents'] = redditTopScores['sentences'].apply(lambda x: [lucem_illud.normalizeTokens(s, stopwordLst = None, stemmer = lucem_illud.stemmer_basic) for s in x])
sents = redditTopScores['normalized_sents']
tagged_sents = [nltk.pos_tag(sent[0]) for sent in sents]

# Get Network

In [141]:
pattern = r'NN*|JJ*'  # you can specify your own pattern to extract name phrases
noun_phrases = [[token for token, tag in tagged_sent if re.match(pattern, tag)] for tagged_sent in tagged_sents ]
edgelist = [edge for phrase in noun_phrases for edge in it.combinations(phrase, 2)]

In [205]:
G = nx.Graph(edgelist)
print ("Total number of nodes(before filtering) in the graph is %s" % len(G))

Total number of nodes(before filtering) in the graph is 348


## We need centrality scores to filter important words
three centralities are applied:
- betweenness centrality
- closeness centrality
- eigenvector centrality

In [206]:
def filterWords(G,filter_ = "betweenness", rule = "number",value_of_rule = 200):
    G = G.copy()
    if filter_ =="betweenness":
        index = nx.betweenness_centrality(G) #betweeness centrality score
    elif filter_ == "closeness":
        index = nx.closeness_centrality(G) #closeness centrality score
    elif filter_ == "eigenvector":
        index = nx.eigenvector_centrality(G) #eigenvector centrality score
    else:
        print("wrong filter paremeter, should be: betweenness/closeness/eigenvector")
        raise
    
        
    if rule=='number':# if filter by limiting the total number of nodes 
        
        sorted_index = sorted(index.items(), key=lambda x:x[1], reverse=True)
        value_of_rule = np.min([value_of_rule, len(G.nodes)])
        
        nodes_remain = {}
        for word, centr in sorted_index[:value_of_rule]:
            nodes_remain[word] = centr
        G.remove_nodes_from([n for n in index if n not in nodes_remain])
        print ("Total number of nodes(after filtering) in the graph is %s" % len(G))
        return G
    
    if rule=='value':# if filter by limiting the value of centrality
        value_of_rule = np.max([float(value_of_rule),0])
        G.remove_nodes_from([n for n in index if index[n] >=value_of_rule])
        print ("Total number of nodes(after filtering) in the graph is %s" % len(G))
        return G

In [207]:
G_1 = filterWords(G, filter_='betweenness',rule='number',value_of_rule=200) #filter by limiting the total # of nodes
G_2 = filterWords(G, filter_='betweenness',rule='value',value_of_rule=0.05) # filter by limiting the value of centrality specified
len(G_1),len(G_2)

Total number of nodes(after filtering) in the graph is 200
Total number of nodes(after filtering) in the graph is 343


(200, 343)